In [6]:
from src import utils

In [ ]:
from src import utils
import importlib
importlib.reload(utils)
#from src.utils import *

print(dir(utils))
print(utils.zip_files_without_structure)

0. Load EFFIS data

In [9]:
# Path to data folders
project_f = r'L:\f02_data\wildfires\spatial_data'

# Create output folder structure
outfolder =  os.path.join(project_f, 'output')
os.makedirs(outfolder, exist_ok=True)

outFIREID_folder =  os.path.join(project_f, 'output', 'FIRE_ID')
os.makedirs(outFIREID_folder, exist_ok=True)

outFIREYEARDOY_folder =  os.path.join(project_f, 'output', 'FIRE_YEARDOY')
os.makedirs(outFIREYEARDOY_folder, exist_ok=True)

In [11]:
# load wildfires data
effis_path =  os.path.join(project_f, 'input', 'effis_rawdata_20240402', 'modis.ba.poly.shp')
effis = gpd.read_file(effis_path)

# read effis id as an integer
effis['id'] = effis['id'].astype(int)

# Convert the column containing dates to datetime format
effis['date'] = pd.to_datetime(effis['FIREDATE'])

# Change the CRS to EPSG:3035
effis = effis.to_crs(epsg=3035)

# Verify the new CRS
print("New CRS:", effis.crs)

New CRS: EPSG:3035


In [ ]:
effis.head()

In [ ]:
# create a lighter subset with target columns
effis_sel = effis[['id', 'COUNTRY',
 'FIREDATE','Year',
 'Month','date','AREA_HA','geometry']]

In [ ]:
#create additional fields to include DOY
effis_sel['doy'] = effis_sel['date'].dt.dayofyear
year2dig = (effis_sel.Year.astype(str).str[-2:])
year000 = year2dig+str('000')
effis_sel['yeardoy'] =year000.astype(int)+ effis_sel['doy'] 
effis_sel.tail()

1. Define reference raster to copy parameters

In [ ]:
# Reference raster
refrast_path = r'M:\f01_dims\ADMbndEEA39v2021\f02_upload_to_jedi\Adm_EEA38_NUTS2021v2022.tif'

# Open the raster data and taking extent and resolution parameters
refraster = rasterio.open(refrast_path)
raster_width = refraster.width
raster_height = refraster.height
minx = refraster.bounds.left
maxy = refraster.bounds.top
resolution = 100

# create list for analysed years
yr_ls = effis_sel.Year.unique().tolist()

2. Rasterize FIREID for each year

In [ ]:
# Loop through years
for yr in yr_ls:    
    effis_yr = effis_sel.query(f'Year == {yr}')
    print(f'year {yr} - nr of fires {effis_yr.id.max()}')
    
    # Start creating one raster per each year using the FIREID
    outfilename = f'FIREID_{yr}.tif'
    outfile = os.path.join(outFIREID_folder, outfilename) 
    if not os.path.exists(outfile):
        rasterize_vector_to_raster(
        effis_yr, 
        'id',
        outfile, 
        raster_width, 
        raster_height, 
        minx, 
        maxy, 
        resolution=resolution, 
        raster_crs='EPSG:3035', 
        chunk_size=1000, 
        compression='lzw'
    )

In [ ]:
from src.utils import *
from src import utils

# zip folder content and save in the same folder
folder_to_zip = outFIREID_folder
output_zip_file = os.path.join(outFIREID_folder, 'FIREID_0023.zip')

utils.zip_files_without_structure(folder_to_zip, output_zip_file)

In [ ]:
print(os.path.join(project_f, 'output', 'FIRE_ID', 'FIREID_0023.zip'))

#### Create LUT

In [13]:
effis23 = effis.query('Year==2023')

In [15]:
effis23.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['id', 'FIREDATE', 'LASTUPDATE', 'COUNTRY', 'PROVINCE', 'COMMUNE',
       'AREA_HA', 'BROADLEA', 'CONIFER', 'MIXED', 'SCLEROPH', 'TRANSIT',
       'OTHERNATLC', 'AGRIAREAS', 'ARTIFSURF', 'OTHERLC', 'PERCNA2K', 'CLASS',
       'Year', 'Month', 'geometry', 'date'],
      dtype='object')>

In [16]:
effis23=effis23[['id', 'FIREDATE', 'LASTUPDATE', 'COUNTRY', 'PROVINCE', 'COMMUNE',
       'AREA_HA', 'Year', 'Month', 'geometry', 'date']]

3. Rasterize FIRE YEARDOY for each year

In [ ]:
# Loop through years
for yr in yr_ls:    
    effis_yr = effis_sel.query(f'Year == {yr}')
    
    # Start creating one raster per each year using the FIREID
    outfilename = f'FIRE_YEARDOY_{yr}.tif'
    outfile = os.path.join(outFIREYEARDOY_folder, outfilename) 
    if not os.path.exists(outfile):
        rasterize_vector_to_raster(
        effis_yr, 
        'yeardoy',
        outfile, 
        raster_width, 
        raster_height, 
        minx, 
        maxy, 
        resolution=resolution, 
        raster_crs='EPSG:3035', 
        chunk_size=1000, 
        compression='lzw'
    )

In [ ]:
# zip folder content and save in the same folder
folder_to_zip = outFIREID_folder
output_zip_file = os.path.join(outFIREYEARDOY_folder, 'FIREYEARDOY_0023.zip')

utils.zip_files_without_structure(folder_to_zip, output_zip_file)